# Escuela Doctoral de Matematica Aplicada
## Manuel A. Sanchez
### Pontificia Universidad Catolica de Chile

# Problema de Poisson
\begin{equation*}
\begin{array}{rclll}
-\Delta u &=& f&\mbox{en}&\Omega:=(0,1)^{2} \\
u &=& u_D& \mbox{sobre}& \partial \Omega_D \\
\nabla u\cdot n&=& g_N&\mbox{sobre} &\partial \Omega_N
\end{array}
\end{equation*}

donde 

\begin{equation*}
\partial \Omega_D:=\{ (0,y)\cup (1,y) \subset \partial \Omega \}, \quad \partial \Omega_N = \partial \Omega \backslash \partial \Omega_D.
\end{equation*}

y para 

\begin{equation*}
\begin{array}{rcl}
f &= &10\exp(-50 ( (x- \frac{1}{2})^{2} + (y - \frac{1}{2})^2)) \\
u_D&=& 1\\
g_N&=& \sin(5x) 
\end{array}
\end{equation*}

Calcule la soluci\'on aproximada para $p=1$ y $h= 0.05$. Presente la gr\'afica de la soluci\'on aproximada.

# NGSolve: Metodo de HDG

###  Importar librerias y modulos

In [8]:
from ngsolve import * # libreria de NGVolve
from netgen.geom2d import unit_square # dominio
from ngsolve.webgui import Draw # comando para graficar en jupyter
from ngsolve.la import EigenValues_Preconditioner

### Dominio y triangulacion del problema

In [9]:
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))

### Resolveremos el problema de Poisson con datos

In [10]:
force = 10*exp(-50*((x-0.5)**2)+(y-0.5)**2)
uD    = 1        # en lado y=0, y=1
gN    = sin(5*x) # en lado x=0, x=1

### Subespacio de elementos finitos, continuas y polinomiales de orden p a trozos

In [11]:
# Space
p = 1
Vh = VectorL2(mesh, order=p)
Wh = L2(mesh, order=p)
Mh = FacetFESpace(mesh, order=p, dirichlet="left|right")

fes = FESpace([Vh,Wh,Mh])

print ("sigmadofs:", fes.Range(0))
print ("udofs:    ", fes.Range(1))
print ("uhatdofs: ", fes.Range(2))

qh, uh, uhat = fes.TrialFunction()
vh, wh, what = fes.TestFunction()

sigmadofs: [0,1380)
udofs:     [1380,2070)
uhatdofs:  [2070,2800)


### Forma bilineal de HDG asociada al problema de Poisson

In [12]:
# stabilization parameter
tau = 1.0
# vector normal
n = specialcf.normal(mesh.dim)
# flujo numerico de HDG
qhatn = qh*n+tau*(uh-uhat)

# resolver solo para uhat
condense=True
a = BilinearForm(fes, condense=condense)
a += (qh*vh - uh*div(vh) - qh*grad(wh))*dx
a += uhat*vh*n*dx(element_boundary=True)
a += qhatn*wh*dx(element_boundary=True)
a += qhatn*what*dx(element_boundary=True)

c = Preconditioner(a, "bddc")
a.Assemble()

### Funcional lineal del problema 

In [13]:
f = LinearForm(fes)
f += force*wh*dx
# boundary term for neumann 
f += -gN*what.Trace()*ds(definedon="top|bottom")
f.Assemble()

### Imponer condicion de Dirichlet

In [14]:
gf = GridFunction(fes)
gfq, gfu, gfuhat = gf.components 
# set Dirichlet boudary condition
gfuhat.Set(uD, BND)

### Resolver el sistema lineal para uhat y recuperar uh y qh

In [19]:
if condense:
    f.vec.data += a.harmonic_extension_trans * f.vec
    
    solvers.CG(mat = a.mat, pre=c.mat, rhs=f.vec, sol=gf.vec, initialize=False)
    
    gf.vec.data += a.harmonic_extension * gf.vec
    gf.vec.data += a.inner_solve * f.vec
    
    preJpoint = a.mat.CreateSmoother(fes.FreeDofs(condense))
    lams = EigenValues_Preconditioner(mat=a.mat, pre=preJpoint)
    print("lambda:", lams)
    condnum = max(lams)/min(lams)
else:

    r = f.vec.CreateVector()
    r.data = f.vec - a.mat * gf.vec
    inv = a.mat.Inverse(freedofs=X.FreeDofs())
    gf.vec.data += inv * r

CG iteration 1, residual = 1.2048355733885812     
CG iteration 2, residual = 0.1418347800735328     
CG iteration 3, residual = 0.008053155002708718     
CG iteration 4, residual = 0.0008944689563761696     
CG iteration 5, residual = 0.00012355841130133095     
CG iteration 6, residual = 3.435667864802125e-05     
CG iteration 7, residual = 1.081972706481021e-05     
CG iteration 8, residual = 1.7375120615792677e-06     
CG iteration 9, residual = 2.6703641667699506e-07     
CG iteration 10, residual = 7.06653227987738e-08     
CG iteration 11, residual = 1.6977348576662445e-08     
CG iteration 12, residual = 3.2106547175336868e-09     
CG iteration 13, residual = 7.078449418309436e-10     
CG iteration 14, residual = 1.2849251059181276e-10     
CG iteration 15, residual = 3.276941132753706e-11     
CG iteration 16, residual = 6.5914868139915924e-12     
CG iteration 17, residual = 1.1614491751476432e-12     
lambda: 


ValueError: max() arg is an empty sequence

### Graficar solucion

In [9]:
Draw(gfu, mesh, "uh")
Draw(gfq[0], mesh, "qh")
#Draw(gfuhat, mesh, "uhat", facet=True)

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

WebGuiWidget(value={'ngsolve_version': '6.2.2203', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene